## Activate virtual environment

In [43]:
%%bash 
source ./MedandEECS_env/bin/activate
pip -V

pip 24.0 from /Users/qwer/files/developer/Medicine_and_EECS/MedandEECS_env/lib/python3.12/site-packages/pip (python 3.12)


## Import packages

In [44]:
import pandas as pd
import numpy as np
import os
from openai import OpenAI
from pydantic import BaseModel


## Initialize client

In [45]:
client  = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Define output format

In [ ]:
class cellTypeFormat(BaseModel):
       cellType: str

## Read cell clusters file 
all.csv

In [ ]:
# TODO read all.csv and into dataframe
def read_cluster(path):
      dataframe = pd.read_csv(path)
      print(dataframe['marker'])
      return dataframe 
gene_list = read_cluster('./Data/all.csv')
print(gene_list.iloc[0])

0       MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
1       MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...
2       IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...
3       IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...
4       GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...
                              ...                        
1125    KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...
1126    JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...
1127    TFF3,PKHD1L1,PROX1,NTS,FLT4,RELN,GPR182,STAB2,...
1128    VWF,ACKR1,PLVAP,PECAM1,AQP1,RAMP3,CLEC14A,SOX1...
1129    MSLN,UPK3B,MUC16,CALB2,KLK11,BICDL1,CHAC1,TGM1...
Name: marker, Length: 1130, dtype: object
dataset                                                               Azimuth
tissue                                                                   PBMC
marker                      MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
manual_annotation                                         Intermediate B cell
manual_C

## Split genelist to generate user messages

In [ ]:
# Generate the user message for each chunk
def generate_user_message(gene_list:  pd.DataFrame, max_chunk_size=100) -> list:
    # Split the gene_list into chunks
    chunks = [group for _, group in gene_list.groupby(gene_list.index // max_chunk_size)]
    messages = []
    for chunk in chunks:
        # Construct the user message for each chunk
        user_message = (
            chunk['tissue'] + ' : '
            + chunk['marker']
        )
        messages.append(user_message)
    return messages

# Generate messages with a tunable chunk size
messages = generate_user_message(gene_list, max_chunk_size=10)  # Adjust max_chunk_size as needed
print(messages[0])

0    PBMC : MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD7...
1    PBMC : MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, ...
2    PBMC : IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, ...
3    PBMC : IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, T...
4    PBMC : GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, G...
5    PBMC : TCF7, CD4, CCR7, IL7R, FHIT, LEF1, MAL,...
6    PBMC : MKI67, TOP2A, PCLAF, CENPF, TYMS, NUSAP...
7    PBMC : IL7R, TMSB10, CD4, ITGB1, LTB, TRAC, AQ...
8    PBMC : IL7R, CCL5, FYB1, GZMK, IL32, GZMA, KLR...
9    PBMC : RTKN2, FOXP3, AC133644.2, CD4, IL2RA, T...
dtype: object


## Upload file

In [ ]:
def upload_cell_cluster(filename):
       global client
       client.files.create(
              file=open(filename, "rb"),
              purpose="assistant"
       )
       print(client.files.list())

## Send prompt to GPT models

In [ ]:
def annotateCell(messages: list)->list:
    global client
    responses = []
    for message in messages:
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content":  "Identify cell types using the following tissue name and markers separately for each row. "
            + "Only provide the cell type name. "
            + "Do not show numbers before the name. "
            + "Some can be a mixture of multiple cell types."},
                {"role": "user", "content": message}
            ],
            response_format=cellTypeFormat,
        )
        responses.append(completion.choices[0].message.parsed)
    return responses
